In [1]:
from math import ceil
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, confusion_matrix

import sklearn
import seaborn as sns
import pandas as pd
import numpy as np


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("C:/datasets/CSGO match/csgo_round_snapshots.csv")
df.head()

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT


In [3]:
df.shape

(122410, 97)

In [4]:
df['round_winner'].value_counts()

T     62406
CT    60004
Name: round_winner, dtype: int64

In [5]:
data = pd.concat((df.drop(["map","bomb_planted"], axis=1), pd.get_dummies(df["map"]), pd.get_dummies(df["bomb_planted"]).rename(columns={False:"Bomb_no", True:"Bomb_yes"})), axis=1)

In [6]:
data["round_winner"] = data["round_winner"].replace("CT", 1)
data["round_winner"] = data["round_winner"].replace("T", 0)

In [7]:
data['round_winner'].value_counts()

0    62406
1    60004
Name: round_winner, dtype: int64

In [8]:
X = data.drop('round_winner',axis=1)
y = data['round_winner'].values

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_X = scaler.fit_transform(X)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2,random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# params = {'n_estimators': [200, 500, 800],
#     'max_features': ['sqrt', 'log2'],
#     'max_depth' : [5,10,15,20],
#     'criterion' :['gini', 'entropy']}
model = RandomForestClassifier(n_estimators=100, max_features='sqrt',max_depth=32, criterion='gini')
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=32)

In [13]:
print(model.score(X_test,y_test))

0.8800751572583939


In [14]:
from sklearn.metrics import confusion_matrix,classification_report

In [15]:
y_pred = model.predict(X_test)

In [16]:
print(confusion_matrix(y_test,y_pred))

[[10890  1588]
 [ 1348 10656]]


In [17]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88     12478
           1       0.87      0.89      0.88     12004

    accuracy                           0.88     24482
   macro avg       0.88      0.88      0.88     24482
weighted avg       0.88      0.88      0.88     24482



In [18]:
X_train = torch.tensor(X_train).float().cuda(); X_train.shape

torch.Size([97928, 104])

In [19]:
y_train = torch.tensor(y_train).view(-1,1).float().cuda(); y_train.shape

torch.Size([97928, 1])

In [20]:
X_test = torch.tensor(X_test).float().cuda(); X_test.shape

torch.Size([24482, 104])

In [21]:
y_test = torch.tensor(y_test).view(-1,1).float().cuda(); y_test.shape

torch.Size([24482, 1])

In [22]:
model = nn.Sequential(nn.Linear(104,512),
                      nn.ReLU(),
                      nn.Linear(512,256),
                      nn.ReLU(),
                      nn.Linear(256,128),
                      nn.ReLU(),
                      nn.Linear(128,64),
                      nn.ReLU(),
                      nn.Linear(64,1),
                      nn.Sigmoid()).to('cuda')

In [23]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
# TEST ACCURACY
probs = model(X_test).cpu().data.numpy()
fpr, tpr, thr = roc_curve(y_test.cpu(), probs)
roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'thr':thr});
roc['random'] = roc['fpr'].copy()
roc['diff'] = roc['tpr'] - roc['fpr']
roc = roc.sort_values('diff', ascending=False).reset_index().drop('index', axis=1)
threshold = roc['thr'].loc[0]
preds_chosen = (probs >= threshold).astype('int64')
preds_auto = (probs >= 0.5).astype('int64')
accuracy_chosen = accuracy_score(y_test.cpu(), preds_chosen)
accuracy_auto = accuracy_score(y_test.cpu(), preds_auto)

if accuracy_chosen > accuracy_auto:
    preds = preds_chosen
elif accuracy_auto > accuracy_chosen:
    preds = preds_auto
    
accuracy_ = accuracy_score(y_test.cpu(), preds)
print(f'accuracy: {accuracy_}') 

accuracy: 0.5096805816518258


In [25]:
# accuracy_min = 0.9035209541704109
# epochs = 500
# BATCH_SIZE = 128
# for e in range(epochs):
# #     optimizer = optim.Adam(model.parameters(), lr=0.0001)
# #     model.load_state_dict(torch.load('model_CSGO.pt'))
#     model.train() 
#     full_steps_of_epoch = ceil(X_train.shape[0] / BATCH_SIZE)
#     prelast_step = int(X_train.shape[0] / BATCH_SIZE)
#     LAST_BATCH = BATCH_SIZE - ((full_steps_of_epoch * BATCH_SIZE) - X_train.shape[0])
#     counter = 0
#     train_loss = 0.0
#     for i in range(full_steps_of_epoch):
#         if counter != prelast_step * BATCH_SIZE:
#             counter += BATCH_SIZE
#             X_batch = X_train[counter-BATCH_SIZE:counter]
#             y_batch = y_train[counter-BATCH_SIZE:counter]
#             optimizer.zero_grad()
#             output = model(X_batch)
#             loss = criterion(output, y_batch)
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         elif counter == prelast_step * BATCH_SIZE:
#             counter += LAST_BATCH
#             X_batch = X_train[counter-LAST_BATCH:counter]
#             y_batch = y_train[counter-LAST_BATCH:counter]
#             optimizer.zero_grad()
#             output = model(X_batch)
#             loss = criterion(output, y_batch)
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
                
#     train_loss = train_loss/full_steps_of_epoch
#     print(f"training loss: {train_loss/full_steps_of_epoch}")
    
    
#     with torch.no_grad():
#         model.eval()
#         loss = criterion(model(X_test), y_test)
#         probs = model(X_test).cpu().data.numpy()
#         fpr, tpr, thr = roc_curve(y_test.cpu(), probs)
#         roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'thr':thr});
#         roc['random'] = roc['fpr'].copy()
#         roc['diff'] = roc['tpr'] - roc['fpr']
#         roc = roc.sort_values('diff', ascending=False).reset_index().drop('index', axis=1)
#         threshold = roc['thr'].loc[0]
#         preds_chosen = (probs >= threshold).astype('int64')
#         preds_auto = (probs >= 0.5).astype('int64')
#         accuracy_chosen = accuracy_score(y_test.cpu(), preds_chosen)
#         accuracy_auto = accuracy_score(y_test.cpu(), preds_auto)

#         if accuracy_chosen > accuracy_auto:
#             preds = preds_chosen
#         elif accuracy_auto > accuracy_chosen:
#             preds = preds_auto

#         accuracy_ = accuracy_score(y_test.cpu(), preds)
  
#         print(f"accuracy_: {accuracy_}")
#         if accuracy_ >= accuracy_min:
#             print('accuracy score increased ({:.6f} --> {:.6f}).  Saving model ...'.format(accuracy_min, accuracy_))
#             torch.save(model.state_dict(), 'model_CSGO.pt')
#             accuracy_min = accuracy_

In [26]:
# LOAD MODEL
model.load_state_dict(torch.load('model_CSGO.pt'))

<All keys matched successfully>

In [27]:
# TEST ACCURACY
probs = model(X_test).cpu().data.numpy()
fpr, tpr, thr = roc_curve(y_test.cpu(), probs)
roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'thr':thr});
roc['random'] = roc['fpr'].copy()
roc['diff'] = roc['tpr'] - roc['fpr']
roc = roc.sort_values('diff', ascending=False).reset_index().drop('index', axis=1)
threshold = roc['thr'].loc[0]
preds_chosen = (probs >= threshold).astype('int64')
preds_auto = (probs >= 0.5).astype('int64')
accuracy_chosen = accuracy_score(y_test.cpu(), preds_chosen)
accuracy_auto = accuracy_score(y_test.cpu(), preds_auto)

if accuracy_chosen > accuracy_auto:
    preds = preds_chosen
elif accuracy_auto > accuracy_chosen:
    preds = preds_auto
    
accuracy_ = accuracy_score(y_test.cpu(), preds)
print(f'accuracy: {accuracy_}') 

accuracy: 0.9035209541704109


In [28]:
probs = model(torch.tensor(X_test).float().cuda()).cpu().data.numpy()

C:\Users\TimurRashitov\AppData\Local\Temp\ipykernel_4228\3971103236.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probs = model(torch.tensor(X_test).float().cuda()).cpu().data.numpy()


In [32]:
(probs > threshold).squeeze().astype('int64')

array([1, 1, 1, ..., 0, 1, 0], dtype=int64)

In [35]:
print(confusion_matrix(y_test.cpu(),(probs > threshold).squeeze().astype('int64')))

[[11357  1121]
 [ 1242 10762]]


In [36]:
print(classification_report(y_test.cpu(),(probs > threshold).squeeze().astype('int64')))

              precision    recall  f1-score   support

         0.0       0.90      0.91      0.91     12478
         1.0       0.91      0.90      0.90     12004

    accuracy                           0.90     24482
   macro avg       0.90      0.90      0.90     24482
weighted avg       0.90      0.90      0.90     24482

